# Battles of Neighborhood - Final Assignment

This Notebook will be used for the data analysis of the neighborhoods in New York to determine the best place to open a vegetarian/vegan restaurant.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## Importation of New York data and creation of dataframes

Import the dataset of New York used in Module three

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods_data = newyork_data['features']

In [5]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [21]:
print('The Dataset has a total of {} borough and {} neighborhood'.format(len(neighborhoods['Borough'].unique()),neighborhoods.shape[0]))

The Dataset has a total of 5 borough and 306 neighborhood


Now let's filter the dataframe to keep only data about Brooklyn. 

In [7]:
Brooklyn = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471
5,Brooklyn,Brighton Beach,40.576825,-73.965094
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438
8,Brooklyn,Flatbush,40.636326,-73.958401
9,Brooklyn,Crown Heights,40.670829,-73.943291


So now that I have the Brooklyn data only, Let's set the foursquare parameters.

## Foursquare data acquisition and analysis 

In [8]:
CLIENT_ID = 'VBGQ0UN4PQTEAJNRAR155JQ0MB2532XOZQV4UWXLA0HTK0LB' # your Foursquare ID
CLIENT_SECRET = 'OZQI4YGQSK1NOG4DUFABKYBMTO4NMKWGPRTFAU3WOEEMYLEL' # your Foursquare Secret
VERSION = '20191227' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VBGQ0UN4PQTEAJNRAR155JQ0MB2532XOZQV4UWXLA0HTK0LB
CLIENT_SECRET:OZQI4YGQSK1NOG4DUFABKYBMTO4NMKWGPRTFAU3WOEEMYLEL


In [40]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [10]:
# create map of Manhattan using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Brooklyn['Latitude'], Brooklyn['Longitude'], Brooklyn['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame(pd.np.empty((0, 3)))
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Venue',
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
brooklyn_venues = getNearbyVenues(names=Brooklyn['Neighborhood'],
                                   latitudes=Brooklyn['Latitude'],
                                   longitudes=Brooklyn['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [ ]:
print(brooklyn_venues.shape)
brooklyn_venues.head(10)

Looking at the data orgonized this way makes it difficult to see exactly what kind of venues are present in the neighborhoods. Let's rearrange the data a little bit to get a better idea of what's what.

In [41]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

The Neighborhood column is placed in the middle of the dataframe here. For an easier analysis we can place it as the first column of the dataframe.

In [16]:
cols = list(brooklyn_onehot.columns.values)
cols.head()

['Yoga Studio',
 'Accessories Store',
 'African Restaurant',
 'American Restaurant',
 'Antique Shop',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Art Gallery',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Dealership',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Baseball Stadium',
 'Basketball Court',
 'Beach',
 'Beer Garden',
 'Beer Store',
 'Big Box Store',
 'Bike Rental / Bike Share',
 'Bike Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Boxing Gym',
 'Breakfast Spot',
 'Brewery',
 'Bridge',
 'Bubble Tea Shop',
 'Buffet',
 'Burger Joint',
 'Bus Line',
 'Bus Station',
 'Bus Stop',
 'Business Service',
 'Butcher',
 'Café',
 'Cajun / Creole Restaurant',
 'Candy Store',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Caucasian Restaurant',
 'Check Cashing Service',
 'Cheese Shop',
 'Chinese Restaurant',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'Concert Hall',
 'Convenience Store'

In [18]:
brooklyn_onehot = brooklyn_onehot[[
 'Neighborhood','Yoga Studio',
 'Accessories Store',
 'African Restaurant',
 'American Restaurant',
 'Antique Shop',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Art Gallery',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Dealership',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Baseball Stadium',
 'Basketball Court',
 'Beach',
 'Beer Garden',
 'Beer Store',
 'Big Box Store',
 'Bike Rental / Bike Share',
 'Bike Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Boxing Gym',
 'Breakfast Spot',
 'Brewery',
 'Bridge',
 'Bubble Tea Shop',
 'Buffet',
 'Burger Joint',
 'Bus Line',
 'Bus Station',
 'Bus Stop',
 'Business Service',
 'Butcher',
 'Café',
 'Cajun / Creole Restaurant',
 'Candy Store',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Caucasian Restaurant',
 'Check Cashing Service',
 'Cheese Shop',
 'Chinese Restaurant',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'Concert Hall',
 'Convenience Store',
 'Cosmetics Shop',
 'Creperie',
 'Cuban Restaurant',
 'Cycle Studio',
 'Dance Studio',
 'Deli / Bodega',
 'Department Store',
 'Dessert Shop',
 'Dim Sum Restaurant',
 'Diner',
 'Discount Store',
 'Distillery',
 'Dive Bar',
 'Dog Run',
 'Donut Shop',
 'Dry Cleaner',
 'Eastern European Restaurant',
 'Electronics Store',
 'Event Service',
 'Event Space',
 'Factory',
 'Falafel Restaurant',
 'Farm',
 'Farmers Market',
 'Fast Food Restaurant',
 'Field',
 'Fish & Chips Shop',
 'Fish Market',
 'Fishing Store',
 'Flower Shop',
 'Food',
 'Food & Drink Shop',
 'Food Court',
 'Food Truck',
 'French Restaurant',
 'Fried Chicken Joint',
 'Frozen Yogurt Shop',
 'Fruit & Vegetable Store',
 'Furniture / Home Store',
 'Gaming Cafe',
 'Garden',
 'Garden Center',
 'Gas Station',
 'Gastropub',
 'Gay Bar',
 'General Entertainment',
 'German Restaurant',
 'Gift Shop',
 'Golf Course',
 'Gourmet Shop',
 'Greek Restaurant',
 'Grocery Store',
 'Gym',
 'Gym / Fitness Center',
 'Gymnastics Gym',
 'Harbor / Marina',
 'Hardware Store',
 'Health & Beauty Service',
 'Health Food Store',
 'History Museum',
 'Home Service',
 'Hookah Bar',
 'Hotel',
 'Hotpot Restaurant',
 'Ice Cream Shop',
 'Indian Restaurant',
 'Indie Theater',
 'Insurance Office',
 'Intersection',
 'Israeli Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Jazz Club',
 'Jewelry Store',
 'Juice Bar',
 'Karaoke Bar',
 'Kids Store',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Laundromat',
 'Laundry Service',
 'Lebanese Restaurant',
 'Lingerie Store',
 'Liquor Store',
 'Lounge',
 'Market',
 'Martial Arts Dojo',
 'Massage Studio',
 'Mediterranean Restaurant',
 "Men's Store",
 'Metro Station',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Miscellaneous Shop',
 'Mobile Phone Shop',
 'Monument / Landmark',
 'Motorcycle Shop',
 'Movie Theater',
 'Moving Target',
 'Museum',
 'Music Venue',
 'Nail Salon',
 'New American Restaurant',
 'Nightclub',
 'Non-Profit',
 'Noodle House',
 'Optical Shop',
 'Organic Grocery',
 'Other Great Outdoors',
 'Other Repair Shop',
 'Outdoors & Recreation',
 'Outlet Store',
 'Paper / Office Supplies Store',
 'Park',
 'Performing Arts Venue',
 'Peruvian Restaurant',
 'Pet Store',
 'Pharmacy',
 'Pie Shop',
 'Piercing Parlor',
 'Pilates Studio',
 'Pizza Place',
 'Playground',
 'Plaza',
 'Polish Restaurant',
 'Pool',
 'Print Shop',
 'Pub',
 'Racetrack',
 'Ramen Restaurant',
 'Record Shop',
 'Rental Car Location',
 'Restaurant',
 'River',
 'Rock Club',
 'Roof Deck',
 'Russian Restaurant',
 'Sake Bar',
 'Salad Place',
 'Salon / Barbershop',
 'Sandwich Place',
 'Scenic Lookout',
 'School',
 'Sculpture Garden',
 'Seafood Restaurant',
 'Shabu-Shabu Restaurant',
 'Shipping Store',
 'Shoe Store',
 'Shopping Mall',
 'Skating Rink',
 'Smoke Shop',
 'Soccer Field',
 'South American Restaurant',
 'Southern / Soul Food Restaurant',
 'Spa',
 'Spanish Restaurant',
 'Sporting Goods Shop',
 'Sports Bar',
 'Steakhouse',
 'Supermarket',
 'Supplement Shop',
 'Surf Spot',
 'Sushi Restaurant',
 'Taco Place',
 'Taiwanese Restaurant',
 'Tapas Restaurant',
 'Tattoo Parlor',
 'Tea Room',
 'Tennis Court',
 'Thai Restaurant',
 'Theater',
 'Theme Park Ride / Attraction',
 'Thrift / Vintage Store',
 'Tibetan Restaurant',
 'Toy / Game Store',
 'Trail',
 'Turkish Restaurant',
 'Used Bookstore',
 'Varenyky restaurant',
 'Vegetarian / Vegan Restaurant',
 'Video Game Store',
 'Video Store',
 'Vietnamese Restaurant',
 'Waterfront',
 'Whisky Bar',
 'Wine Bar',
 'Wine Shop',
 'Wings Joint',
 "Women's Store"]]

In [19]:
brooklyn_onehot.head(10)

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

Now we have all the venues for Brooklyn's Neighborhoods. As described in my the inroduction, the goal is to find a neighborhood where there is not already to much vegetarian/vegan restaurants but also other venues that could attract the clientele that we are targeting. 
So let's take a look at what are the types of venues that are the most correlated with vegetarian and vegan restaurants.

In [20]:
# we sum all venues by Neighborhood to make the dataframe easier to read
bk = brooklyn_onehot.groupby('Neighborhood').sum().reset_index() 
bk.head()

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bath Beach,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1

In [21]:
import pandas as pd
import numpy as np

shape = (50, 4460)

data = np.random.normal(size=shape)

data[:, 1000] += data[:, 2000]

c = bk.corr().abs()

s = c.unstack()
so = s.sort_values()

vg = pd.DataFrame(data = so)
vg1 = vg.reset_index()
vg1.head()

,level_0,level_1,0
0,Gym / Fitness Center,Thrift / Vintage Store,2.223039e-18
1,Thrift / Vintage Store,Gym / Fitness Center,2.223039e-18
2,Convenience Store,Fast Food Restaurant,4.018751e-18
3,Fast Food Restaurant,Convenience Store,4.018751e-18
4,Cocktail Bar,Spanish Restaurant,6.325619e-18


In [22]:
vegetarian = ['Vegetarian / Vegan Restaurant']
vg1 = vg1[~vg1.level_0.isin(vegetarian) & 
           vg1.level_1.isin(vegetarian)]
vg2 = vg1.rename(columns={'level_0': 'var1', 'level_1': 'var2',0: 'corr'})
vg2

,var1,var2,corr
60,Yoga Studio,Vegetarian / Vegan Restaurant,0.001416
7800,Bagel Shop,Vegetarian / Vegan Restaurant,0.017985
8147,Deli / Bodega,Vegetarian / Vegan Restaurant,0.018985
8506,Sandwich Place,Vegetarian / Vegan Restaurant,0.019715
15507,Tea Room,Vegetarian / Vegan Restaurant,0.025474
15508,Sculpture Garden,Vegetarian / Vegan Restaurant,0.025474
15514,Tibetan Restaurant,Vegetarian / Vegan Restaurant,0.025474
15522,Shabu-Shabu Restaurant,Vegetarian / Vegan Restaurant,0.025474
15540,Caucasian Restaurant,Vegetarian / Vegan Restaurant,0.025474
15542,Mediterranean Restaurant,Vegetarian / Vegan Restaurant,0.025474


In [23]:
vg2 = vg2.sort_values(by='corr', ascending=False)
print(vg2.shape)

(244, 3)


Ok so now we have all correlation values with vegatarien and vegan restaurants. Let's take all venue categories with a correlation value higher than 0.2.

In [24]:
vg2.dtypes

var1     object
var2     object
corr    float64
dtype: object

In [25]:
vg_corr = vg2[vg2['corr'] > 0.2]
print(vg_corr.shape)
vg_corr

(28, 3)


,var1,var2,corr
59608,Gymnastics Gym,Vegetarian / Vegan Restaurant,0.810471
59167,Indie Theater,Vegetarian / Vegan Restaurant,0.568921
59163,Taiwanese Restaurant,Vegetarian / Vegan Restaurant,0.568921
59145,Lebanese Restaurant,Vegetarian / Vegan Restaurant,0.568921
59142,Laundry Service,Vegetarian / Vegan Restaurant,0.568921
59127,Gay Bar,Vegetarian / Vegan Restaurant,0.568921
59116,Polish Restaurant,Vegetarian / Vegan Restaurant,0.568921
59109,Nightclub,Vegetarian / Vegan Restaurant,0.568921
59096,Used Bookstore,Vegetarian / Vegan Restaurant,0.568921
58977,Concert Hall,Vegetarian / Vegan Restaurant,0.533997


As we can see, the only venue with a significant correlation to vegetarian and vegan restaurants are gymnastic gym. All other venue have only a correlation of 0.5 at best or less. So correlation might not be the best way to select target values in this case. 

Let's instead explore the top 5 venues of all the Neighborhoods in Old Toronto to have a better idea of what we are dealing with. 

In [26]:
num_top_venues = 5

for hood in bk['Neighborhood']:
    print("----"+hood+"----")
    temp = bk[bk['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0    Chinese Restaurant   2.0
1    Italian Restaurant   2.0
2              Pharmacy   2.0
3       Bubble Tea Shop   2.0
4  Fast Food Restaurant   2.0


----Bay Ridge----
                 venue  freq
0                  Spa   3.0
1        Grocery Store   2.0
2           Hookah Bar   2.0
3  American Restaurant   2.0
4     Greek Restaurant   2.0


----Bedford Stuyvesant----
           venue  freq
0  Deli / Bodega   3.0
1           Café   2.0
2    Coffee Shop   2.0
3    Pizza Place   2.0
4            Bar   2.0


----Bensonhurst----
                venue  freq
0  Chinese Restaurant   4.0
1  Italian Restaurant   3.0
2    Sushi Restaurant   2.0
3          Donut Shop   2.0
4      Ice Cream Shop   2.0


----Bergen Beach----
                venue  freq
0     Harbor / Marina   2.0
1      Baseball Field   1.0
2          Playground   1.0
3          Donut Shop   1.0
4  Athletics & Sports   1.0


----Boerum Hill----
                    venue  freq
0    

Let's put all of this data into a data frame but taking into account not 5 but 15 top venues instead.

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bk['Neighborhood']

for ind in np.arange(bk.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bk.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Bath Beach,Italian Restaurant,Pharmacy,Bubble Tea Shop,Fast Food Restaurant,Chinese Restaurant,Hookah Bar,Burger Joint,Cantonese Restaurant,Restaurant,Rental Car Location,Sushi Restaurant,Coffee Shop,Donut Shop,Park,Diner
1,Bay Ridge,Spa,Grocery Store,Hookah Bar,American Restaurant,Pizza Place,Greek Restaurant,Coffee Shop,Tea Room,Taco Place,Sushi Restaurant,Chinese Restaurant,Bookstore,Sports Bar,Juice Bar,Lounge
2,Bedford Stuyvesant,Deli / Bodega,Bar,Café,Pizza Place,Coffee Shop,BBQ Joint,Fruit & Vegetable Store,Boutique,New American Restaurant,Cocktail Bar,Gourmet Shop,Juice Bar,Bagel Shop,Thrift / Vintage Store,Fried Chicken Joint
3,Bensonhurst,Chinese Restaurant,Italian Restaurant,Ice Cream Shop,Donut Shop,Sushi Restaurant,Flower Shop,Butcher,Shabu-Shabu Restaurant,Factory,Noodle House,Dance Studio,Russian Restaurant,Cosmetics Shop,Park,Pet Store
4,Bergen Beach,Harbor / Marina,Donut Shop,Playground,Athletics & Sports,Baseball Field,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Women's Store,Fish Market,Fishing Store,Flower Shop,Food
5,Boerum Hill,Bar,Yoga Studio,Coffee Shop,Spa,Furniture / Home Store,Antique Shop,Grocery Store,Sandwich Place,Middle Eastern Restaurant,Martial Arts Dojo,Chinese Restaurant,Cocktail Bar,Concert Hall,Sushi Restaurant,Kids Store
6,Borough Park,Bank,Pizza Place,Deli / Bodega,Pharmacy,Restaurant,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Café,Liquor Store,Hotel,American Restaurant,Gym / Fitness Center,Electronics Store,Fish & Chips Shop
7,Brighton Beach,Sushi Restaurant,Russian Restaurant,Restaurant,Non-Profit,Other Great Outdoors,Supplement Shop,Lounge,Taco Place,Bookstore,Beach,Korean Restaurant,Eastern European Restaurant,Bank,Pharmacy,Donut Shop
8,Broadway Junction,Donut Shop,Diner,Sandwich Place,Gas Station,Metro Station,Grocery Store,Bus Station,Bus Stop,Fried Chicken Joint,Burger Joint,Discount Store,Ice Cream Shop,Caribbean Restaurant,Flower Shop,Farmers Market
9,Brooklyn Heights,Yoga Studio,Pet Store,Park,Scenic Lookout,Coffee Shop,Cosmetics Shop,Playground,Pizza Place,Pilates Studio,Deli / Bodega,Diner,Sushi Restaurant,Mobile Phone Shop,Mexican Restaurant,Garden



## Creation of clusters for Brooklyn Neighborhoods

Ok, we have all our top venues in all the Neighborhhods of Brooklyn. Now the goal is to see if we can seperate them in cluster with the k-nearest neighbor machine learning method. If we get different clusters we can analyse what kind of neighborhood it contains and exclude some of them for our new restaurant. 

In [29]:
# set number of clusters, let's start with five and then expand to test 
kclusters = 6

brooklyn_clustering = bk.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 3, 2, 2, 3, 1, 2, 2, 5], dtype=int32)

In [30]:
# add clustering labels to the dataframe 
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

brooklyn_merged = Brooklyn

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,3,Spa,Grocery Store,Hookah Bar,American Restaurant,Pizza Place,Greek Restaurant,Coffee Shop,Tea Room,Taco Place,Sushi Restaurant,Chinese Restaurant,Bookstore,Sports Bar,Juice Bar,Lounge
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2,Chinese Restaurant,Italian Restaurant,Ice Cream Shop,Donut Shop,Sushi Restaurant,Flower Shop,Butcher,Shabu-Shabu Restaurant,Factory,Noodle House,Dance Studio,Russian Restaurant,Cosmetics Shop,Park,Pet Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,4,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Bank,Mobile Phone Shop,Fried Chicken Joint,Women's Store,Pharmacy,Creperie,Record Shop,Supplement Shop,Breakfast Spot,Grocery Store,Gym
3,Brooklyn,Greenpoint,40.730201,-73.954241,3,Bar,Café,Mexican Restaurant,Spa,Cocktail Bar,Coffee Shop,French Restaurant,Boutique,Bookstore,Furniture / Home Store,Liquor Store,Sushi Restaurant,Beer Store,Laundry Service,Gastropub
4,Brooklyn,Gravesend,40.595260,-73.973471,3,Pizza Place,Bakery,Bus Station,Lounge,Italian Restaurant,Deli / Bodega,Martial Arts Dojo,Men's Store,Furniture / Home Store,Fish Market,Baseball Field,Farmers Market,Chinese Restaurant,Donut Shop,Record Shop


Let's create a map to see how the clusters of the Neighbothoods are spread in Odl Toronto. 

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now let's examine the clusters:

__Cluster 1__

In [34]:
brooklyn_merged.loc[brooklyn_merged['Cluster_Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Boat or Ferry,Karaoke Bar,Diner,Pizza Place,Deli / Bodega,Creperie,Outlet Store,Restaurant,Chinese Restaurant,Russian Restaurant,Café,Fishing Store


__Cluster 2__

In [35]:
brooklyn_merged.loc[brooklyn_merged['Cluster_Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
34,Borough Park,Bank,Pizza Place,Deli / Bodega,Pharmacy,Restaurant,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Café,Liquor Store,Hotel,American Restaurant,Gym / Fitness Center,Electronics Store,Fish & Chips Shop
48,Georgetown,Bank,Pharmacy,Breakfast Spot,Donut Shop,Women's Store,Shipping Store,Frozen Yogurt Shop,Burger Joint,Shopping Mall,Mexican Restaurant,Sandwich Place,Miscellaneous Shop,Supplement Shop,Clothing Store,Coffee Shop
60,Mill Basin,Pizza Place,Bank,Japanese Restaurant,Bagel Shop,Pharmacy,Supermarket,Breakfast Spot,Sushi Restaurant,Frozen Yogurt Shop,Food Truck,Cosmetics Shop,Gift Shop,Burger Joint,Middle Eastern Restaurant,Bus Station
66,Homecrest,Bank,Mexican Restaurant,Chinese Restaurant,Pizza Place,Sandwich Place,Donut Shop,Pharmacy,Other Repair Shop,Sushi Restaurant,Lounge,Tattoo Parlor,Fast Food Restaurant,Mobile Phone Shop,Bakery,Cosmetics Shop


__Cluster 3__

In [36]:
brooklyn_merged.loc[brooklyn_merged['Cluster_Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,Bensonhurst,Chinese Restaurant,Italian Restaurant,Ice Cream Shop,Donut Shop,Sushi Restaurant,Flower Shop,Butcher,Shabu-Shabu Restaurant,Factory,Noodle House,Dance Studio,Russian Restaurant,Cosmetics Shop,Park,Pet Store
5,Brighton Beach,Sushi Restaurant,Russian Restaurant,Restaurant,Non-Profit,Other Great Outdoors,Supplement Shop,Lounge,Taco Place,Bookstore,Beach,Korean Restaurant,Eastern European Restaurant,Bank,Pharmacy,Donut Shop
9,Crown Heights,Pizza Place,Bookstore,Museum,Café,Salon / Barbershop,Burger Joint,Candy Store,Bakery,Bagel Shop,Coffee Shop,Pharmacy,Supermarket,Moving Target,Deli / Bodega,Bus Station
10,East Flatbush,Print Shop,Supermarket,Pharmacy,Park,Chinese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Moving Target,Food & Drink Shop,Department Store,Hardware Store,Eastern European Restaurant,Fish & Chips Shop,Field,Health & Beauty Service
12,Windsor Terrace,Diner,Café,Park,Deli / Bodega,Grocery Store,Thrift / Vintage Store,Middle Eastern Restaurant,French Restaurant,Salad Place,Bookstore,Chinese Restaurant,Sushi Restaurant,Beer Store,Coffee Shop,Bar
14,Brownsville,Park,Chinese Restaurant,Moving Target,Restaurant,Pizza Place,Caribbean Restaurant,Fried Chicken Joint,Discount Store,Trail,Farmers Market,Performing Arts Venue,Spanish Restaurant,Pharmacy,Pool,Deli / Bodega
24,Park Slope,Bagel Shop,American Restaurant,Coffee Shop,Furniture / Home Store,Japanese Restaurant,Yoga Studio,Frozen Yogurt Shop,Bookstore,Sporting Goods Shop,Burger Joint,Dessert Shop,Kids Store,Falafel Restaurant,Organic Grocery,Pet Store
26,East New York,Deli / Bodega,Fast Food Restaurant,Gym,Pizza Place,Convenience Store,Chinese Restaurant,Event Service,Caribbean Restaurant,Music Venue,Metro Station,Plaza,Asian Restaurant,History Museum,Fish Market,Fish & Chips Shop
27,Starrett City,Supermarket,Pizza Place,American Restaurant,Moving Target,Pharmacy,Caribbean Restaurant,Bus Stop,River,Donut Shop,Women's Store,Field,Farm,Farmers Market,Fast Food Restaurant,Fish Market
28,Canarsie,Deli / Bodega,Caribbean Restaurant,Gym,Bus Line,Asian Restaurant,Field,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Factory,Fish Market,Fishing Store,Flower Shop,Food


__Cluster 4__

In [37]:
brooklyn_merged.loc[brooklyn_merged['Cluster_Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Bay Ridge,Spa,Grocery Store,Hookah Bar,American Restaurant,Pizza Place,Greek Restaurant,Coffee Shop,Tea Room,Taco Place,Sushi Restaurant,Chinese Restaurant,Bookstore,Sports Bar,Juice Bar,Lounge
3,Greenpoint,Bar,Café,Mexican Restaurant,Spa,Cocktail Bar,Coffee Shop,French Restaurant,Boutique,Bookstore,Furniture / Home Store,Liquor Store,Sushi Restaurant,Beer Store,Laundry Service,Gastropub
4,Gravesend,Pizza Place,Bakery,Bus Station,Lounge,Italian Restaurant,Deli / Bodega,Martial Arts Dojo,Men's Store,Furniture / Home Store,Fish Market,Baseball Field,Farmers Market,Chinese Restaurant,Donut Shop,Record Shop
13,Prospect Heights,Bar,Thai Restaurant,Café,Cocktail Bar,Ice Cream Shop,Latin American Restaurant,Sushi Restaurant,Garden Center,Diner,Mexican Restaurant,Business Service,Yoga Studio,Korean Restaurant,Caribbean Restaurant,Nail Salon
15,Williamsburg,Coffee Shop,Bar,Pizza Place,Bagel Shop,Yoga Studio,Steakhouse,Latin American Restaurant,Tapas Restaurant,Taco Place,Liquor Store,Breakfast Spot,Lounge,Burger Joint,Middle Eastern Restaurant,Event Space
16,Bushwick,Bar,Coffee Shop,Mexican Restaurant,Pizza Place,Thrift / Vintage Store,French Restaurant,Café,Sandwich Place,Nightclub,Latin American Restaurant,Korean Restaurant,Bakery,Bagel Shop,Cuban Restaurant,Used Bookstore
17,Bedford Stuyvesant,Deli / Bodega,Bar,Café,Pizza Place,Coffee Shop,BBQ Joint,Fruit & Vegetable Store,Boutique,New American Restaurant,Cocktail Bar,Gourmet Shop,Juice Bar,Bagel Shop,Thrift / Vintage Store,Fried Chicken Joint
19,Cobble Hill,Italian Restaurant,Ice Cream Shop,Playground,Bar,Cocktail Bar,Yoga Studio,Thai Restaurant,Fish Market,Men's Store,French Restaurant,Deli / Bodega,Boutique,Spa,Spanish Restaurant,Bookstore
20,Carroll Gardens,Spa,Coffee Shop,Bar,Italian Restaurant,Cosmetics Shop,Gourmet Shop,Café,Farmers Market,Gaming Cafe,Restaurant,Cocktail Bar,Beer Garden,Latin American Restaurant,Thai Restaurant,Bakery
21,Red Hook,Bar,Flower Shop,Ice Cream Shop,Seafood Restaurant,American Restaurant,New American Restaurant,Bagel Shop,Café,Brewery,Sandwich Place,Farm,Event Space,Gift Shop,Grocery Store,Bakery


__Cluster 5__

In [38]:
brooklyn_merged.loc[brooklyn_merged['Cluster_Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
2,Sunset Park,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Bank,Mobile Phone Shop,Fried Chicken Joint,Women's Store,Pharmacy,Creperie,Record Shop,Supplement Shop,Breakfast Spot,Grocery Store,Gym
7,Manhattan Terrace,Pizza Place,Donut Shop,Ice Cream Shop,Bagel Shop,Coffee Shop,Liquor Store,Laundromat,Steakhouse,Convenience Store,Bank,Bakery,Grocery Store,Cosmetics Shop,Jazz Club,Japanese Restaurant
8,Flatbush,Pharmacy,Deli / Bodega,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Mexican Restaurant,Juice Bar,Bagel Shop,Donut Shop,Sandwich Place,Pizza Place,Middle Eastern Restaurant,Lounge,Bank,Plaza
11,Kensington,Grocery Store,Thai Restaurant,Pizza Place,Ice Cream Shop,Donut Shop,Bakery,Nail Salon,Sandwich Place,Furniture / Home Store,Spa,Restaurant,Mobile Phone Shop,Coffee Shop,Japanese Restaurant,Bagel Shop
25,Cypress Hills,Fried Chicken Joint,Pizza Place,Spanish Restaurant,Ice Cream Shop,Fast Food Restaurant,Latin American Restaurant,Donut Shop,Supermarket,Baseball Field,Bank,Chinese Restaurant,Metro Station,Discount Store,South American Restaurant,Dance Studio
44,City Line,Donut Shop,Shoe Store,Fried Chicken Joint,Liquor Store,Bakery,Flower Shop,Mobile Phone Shop,Food,Bus Stop,South American Restaurant,Food Truck,Fast Food Restaurant,Supermarket,Lingerie Store,Bank
47,Prospect Park South,Mobile Phone Shop,Caribbean Restaurant,Grocery Store,Latin American Restaurant,Mexican Restaurant,Miscellaneous Shop,Sporting Goods Shop,Department Store,Southern / Soul Food Restaurant,Women's Store,Theater,Fast Food Restaurant,Sandwich Place,Electronics Store,Clothing Store
54,Ditmas Park,Pizza Place,Pharmacy,Caribbean Restaurant,Women's Store,Grocery Store,Ramen Restaurant,Mobile Phone Shop,Mexican Restaurant,Latin American Restaurant,Japanese Restaurant,Health Food Store,Gym / Fitness Center,Farmers Market,Tapas Restaurant,Donut Shop
56,Rugby,Caribbean Restaurant,Bank,Pizza Place,Supermarket,Seafood Restaurant,Chinese Restaurant,Farmers Market,Fast Food Restaurant,Grocery Store,Sandwich Place,Fried Chicken Joint,Pharmacy,Diner,Wings Joint,Mobile Phone Shop
69,Erasmus,Caribbean Restaurant,Grocery Store,Yoga Studio,Bank,Pharmacy,Convenience Store,Donut Shop,School,Music Venue,Mobile Phone Shop,Playground,Food Truck,Fried Chicken Joint,Supermarket,Furniture / Home Store


__Cluster 6__

In [39]:
brooklyn_merged.loc[brooklyn_merged['Cluster_Labels'] == 5, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
18,Brooklyn Heights,Yoga Studio,Pet Store,Park,Scenic Lookout,Coffee Shop,Cosmetics Shop,Playground,Pizza Place,Pilates Studio,Deli / Bodega,Diner,Sushi Restaurant,Mobile Phone Shop,Mexican Restaurant,Garden
38,Clinton Hill,Thai Restaurant,Wine Shop,Yoga Studio,Italian Restaurant,Restaurant,Pizza Place,Diner,Convenience Store,Pub,Pet Store,Chinese Restaurant,Optical Shop,Deli / Bodega,Sandwich Place,Cajun / Creole Restaurant
65,Dumbo,Yoga Studio,Art Gallery,Boxing Gym,Bakery,Gym,Coffee Shop,Antique Shop,Bookstore,Wine Shop,Climbing Gym,Deli / Bodega,Furniture / Home Store,Beach,Salad Place,Sandwich Place


After testing several numbers of clusters, 6 gives the least number of clusters with only one neighborhood inside. 

Now that we have our cluster let's analyze them a little. 

Since our goal is to open a new vegetarian/vegan restaurants and there seems to be no correlations high enough to matter with other types of venues, we are going to use logic and general knowledge to choose our neighborhood based on the clusters we just obtained.

People that are vegetarians or vegans tend to like certain lifestyles and the other way around, people that like certain activities like vegetarian food more than others.
On this note, let's formulate the hypothesis that people that practice yoga tends to be more animal friendly in their way of eating. People comming from an artistic background are more enclined to be vegetarians and vegans. Also, people doing physical activities and being more attentive to their physic and health tend to like more vegetarian and vegan food, even if they are not completely vegans or vegetarians. 

One of the clusters obtained is standing out, cluster number 6. It contains only three neighborhoods but all three of them have a yoga studio as one of the five top venues. All three neighborhoods are also popular based on other sport venues such as pilates studios, gym, parks (jogging) and art venues are also present there. It seems those three neighborhoods are the best ones in thes clusters that we obtained. 

Now, taking all three separately let's analyze them a little more: 
1. Brooklyn Heghts : The most popular venue on this neighborhood is a Yoga Studio, which fits perfectly with our prior hypothesis that yogis are more incline to eat vegetarian or vegan food. So this is a really good point. In the first five venues we also have a park, which is a good place for jogging and other outside activities, a coffee shop, a scenic lookout and a pet store. A pilates studio is one of the ten most popular venues of the neighborhood  which is a good point for our restaurant. An important point is that in the 15 top venues of the neighborhood, only three are restaurants and none are vegetarian or vegan restaurant, meening that this type of clientele is not yet answered for. 
2. Clinton Hills : the yoga studio comes as the third top venue of the neighborhood.  but on the other four, three are restaurants, meening there is already a lot of offer for food in this area. Looking at the rest of the top 15 venues, 8 of the venues in total are restaurants and without other types of venues that we are targeting here. 
3. Dumbo : As Brooklyn Heights, the top venues of the neighbrhood is a yoga studio. The second one is an Art gallery, which is in our top target venues for the neighborhood. In the top ten stands also an Antique Store and a Bookstore, that can be good venues for us too, attracting artistic and hipster people. A Booking Gym, Gym and a Climbing Gym are present in the area, matching our sport target. Finaly, in this neighborhood, the only options for food in the top venues are a Bakery, a Salad Place and a Sandwich place. Those are more food in the run that actual restaurant, leaving the door open for a new one. 

Based on this analysis of the three neighborhoods in our 6th cluster, we can choose Dumbo as the most suitable Neighborhood in Brooklyn to open a new Vegetarian/Vegan restaurant. 

## Conclusion 

Using Foursquare data, we were able to gather the top 15 venues of all neighborhoods of Brokklyn, NY. After analysis, it seems the types there is no real correlation between the presence of venues with the presence of vegetarian/vegan restaurants in Brooklyn. When the data is clustered by nearest neighbor, we can see that some types of Neighborhoods are more suitable than others and with further analysis, the neighborhood Dumbo was chosen as the most suitable for the openning of a new Vegetarian/vegan restaurant. 